In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vehicle-dataset-from-cardekho/car data.csv
/kaggle/input/vehicle-dataset-from-cardekho/car details v4.csv
/kaggle/input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv
/kaggle/input/vehicle-dataset-from-cardekho/Car details v3.csv


### Import the Libraries

In [12]:
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm

### Load the DataSet

In [13]:
df=pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')

#### Check the first five column of dataset.

In [14]:
df.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### Check the total numbers of Rows and Columns in our data.

In [15]:
df.shape

(301, 9)

### Check the Datatype of each column.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


### Check the Null Value in Dataset.

In [17]:
df.isnull().sum()

Car_Name         0
Year             0
Selling_Price    0
Present_Price    0
Kms_Driven       0
Fuel_Type        0
Seller_Type      0
Transmission     0
Owner            0
dtype: int64

### Convert categorical variables to numerical

In [18]:
label_encoders = {}
categorical_cols = ['Fuel_Type', 'Seller_Type', 'Transmission']
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    df[col] = label_encoders[col].fit_transform(df[col])

In [20]:
#Drop 'Car_Name' as it is non-numeric and not useful for regression
df = df.drop(['Car_Name'], axis=1)

In [21]:
# Define independent (X) and dependent (y) variables
X = df.drop(['Selling_Price'], axis=1)  # Independent variables
y = df['Selling_Price']  # Dependent variable

 ### Create Polynomial Features

In [22]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)
X_poly_df = pd.DataFrame(X_poly, columns=poly.get_feature_names_out(X.columns))

# Add constant for statsmodels
X_poly_df = sm.add_constant(X_poly_df)


## Feature Selection Methods

#### (a) Backward Elimination

In [23]:
X_opt = X_poly_df.copy()  # Start with all features
while True:
    model = sm.OLS(y, X_opt).fit()
    p_values = model.pvalues[1:]  # Ignore intercept
    max_p = p_values.max()
    
    if max_p > 0.05:  # If p-value > 0.05, remove the worst feature
        worst_feature = p_values.idxmax()
        X_opt = X_opt.drop(columns=[worst_feature])
    else:
        break

model_backward = sm.OLS(y, X_opt).fit()
print(model_backward.summary())


                            OLS Regression Results                            
Dep. Variable:          Selling_Price   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     900.9
Date:                Sun, 09 Mar 2025   Prob (F-statistic):          1.02e-232
Time:                        14:15:42   Log-Likelihood:                -322.03
No. Observations:                 301   AIC:                             678.1
Df Residuals:                     284   BIC:                             741.1
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

#### (b) Forward Selection

In [24]:
features = X_poly_df.columns.tolist()
selected_features = ['const']
remaining_features = features[1:]

for _ in range(len(remaining_features)):
    best_r2 = -1
    best_feature = None
    
    for feature in remaining_features:
        temp_features = selected_features + [feature]
        model = sm.OLS(y, X_poly_df[temp_features]).fit()
        if model.rsquared > best_r2:
            best_r2 = model.rsquared
            best_feature = feature
    
    if best_feature:
        selected_features.append(best_feature)
        remaining_features.remove(best_feature)

model_forward = sm.OLS(y, X_poly_df[selected_features]).fit()
print(model_forward.summary())


                            OLS Regression Results                            
Dep. Variable:          Selling_Price   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     465.9
Date:                Sun, 09 Mar 2025   Prob (F-statistic):          6.77e-215
Time:                        14:16:36   Log-Likelihood:                -313.74
No. Observations:                 301   AIC:                             691.5
Df Residuals:                     269   BIC:                             810.1
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

#### (c) Bidirectional Selection

In [25]:
X_bi = X_poly_df.copy()
selected_features = X_bi.columns.tolist()

while True:
    # Backward Step: Remove the least significant feature
    model = sm.OLS(y, X_bi).fit()
    p_values = model.pvalues[1:]  # Ignore intercept
    max_p = p_values.max()
    
    if max_p > 0.05:
        worst_feature = p_values.idxmax()
        selected_features.remove(worst_feature)
        X_bi = X_bi[selected_features]
    else:
        break

# Forward Step: Add features if they improve the model
remaining_features = [col for col in X_poly_df.columns if col not in selected_features]

for feature in remaining_features:
    temp_features = selected_features + [feature]
    model = sm.OLS(y, X_poly_df[temp_features]).fit()
    
    if model.pvalues[feature] < 0.05:  # Add if p-value < 0.05
        selected_features.append(feature)

model_bidirectional = sm.OLS(y, X_poly_df[selected_features]).fit()
print(model_bidirectional.summary())


                            OLS Regression Results                            
Dep. Variable:          Selling_Price   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     857.7
Date:                Sun, 09 Mar 2025   Prob (F-statistic):          3.89e-232
Time:                        14:16:50   Log-Likelihood:                -319.82
No. Observations:                 301   AIC:                             675.6
Df Residuals:                     283   BIC:                             742.4
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

#### (d) Keep All Variables

In [26]:
model_all = sm.OLS(y, X_poly_df).fit()
print(model_all.summary())


                            OLS Regression Results                            
Dep. Variable:          Selling_Price   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     465.9
Date:                Sun, 09 Mar 2025   Prob (F-statistic):          6.77e-215
Time:                        14:17:06   Log-Likelihood:                -313.74
No. Observations:                 301   AIC:                             691.5
Df Residuals:                     269   BIC:                             810.1
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

### Compare Slopes/Coefficients

In [27]:
coefs_df = pd.DataFrame({
    "Backward Elimination": model_backward.params,
    "Forward Selection": model_forward.params,
    "Bidirectional Selection": model_bidirectional.params,
    "All Variables": model_all.params
})

print(coefs_df)

                            Backward Elimination  Forward Selection  \
Fuel_Type                           5.000611e+02       4.877890e+02   
Fuel_Type Owner                              NaN      -5.474787e-01   
Fuel_Type Seller_Type                        NaN       8.822820e-01   
Fuel_Type Transmission             -6.481315e-01      -1.145583e+00   
Fuel_Type^2                                  NaN      -3.220753e-01   
Kms_Driven                          6.809315e-03       6.288009e-03   
Kms_Driven Fuel_Type               -2.558749e-05      -2.471733e-05   
Kms_Driven Owner                             NaN      -7.469731e-07   
Kms_Driven Seller_Type                       NaN      -4.607689e-06   
Kms_Driven Transmission                      NaN      -7.518166e-07   
Kms_Driven^2                       -2.828094e-11      -2.166928e-11   
Owner                                        NaN       1.507847e+01   
Owner^2                                      NaN       1.297907e+01   
Presen

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
